# Selecting Data from PostgreSQL

## Introduction

Now that you've gotten a brief introduction to SQL, its time to get some hands on practice connecting to a database via Python and executing to some queries.

## Objectives

You will be able to:

* Understand the basic structure of a `SELECT` statement in SQL

## Connecting To a Database

First let's connect to our database by importing psycopg2 and running the following cell in our notebook. You'll need a cursor object (`cur`) to fetch results. Cursor objects allow you to keep track of which result set is which, since it's possible to run multiple queries before you're done fetching the results of the first.

```python 
import psycopg2
conn = psycopg2.connect('data_base_infos')
cur = conn.cursor()
```
you will not have `psycog2` already installed in your anaconda enviroment, so go ahead and execute the following to install pyscog2 to your `(base)(Python3.x.x)`

```python
$ conda activate base 
$ pip install psycopg2
$ conda deactivate
$ conda list  ** to check
```

In [174]:
# connect database and create cursor here
import psycopg2 
conn = psycopg2.connect(
    host = "localhost",
    database="users",
    user="postgres",
    password="postgres",
    port="5432"
)
cur = conn.cursor()

## Querying Via the Connection

Now that you're connected to the database, let's take a look at how you can query the data within.

With your cursor object, you can execute queries

In [175]:
cur.execute('SELECT * FROM aws')

In [164]:
cur.description

(Column(name='article_id', type_code=23),
 Column(name='author_id', type_code=23),
 Column(name='viewer_id', type_code=23),
 Column(name='view_date', type_code=1082))

In [165]:
type(cur)

psycopg2.extensions.cursor

The execute command itself only returns the cursor object. To see the results, you must use the fetchall method afterwards.


In [168]:
len(cur.fetchall())

0

It's also possible to combine the previous two cells into one line, like so:

In [169]:
## Uncomment cell to display contents:

cur.execute("SELECT * FROM aws")
cur.fetchall()

[(1, 3, 5, datetime.date(2019, 8, 1)),
 (1, 3, 6, datetime.date(2019, 8, 2)),
 (2, 7, 6, datetime.date(2019, 8, 2)),
 (3, 4, 4, datetime.date(2019, 7, 21)),
 (3, 4, 4, datetime.date(2019, 7, 21)),
 (5, 5, 5, datetime.date(2019, 7, 24)),
 (1, 1, 3, datetime.date(2019, 8, 1)),
 (1, 1, 3, datetime.date(2019, 8, 1)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2020, 8, 7)),
 (10, 11, 12, datetime.date(2020, 8, 7))]

In [176]:
data = cur.fetchall()

In [177]:
data

[(1, 3, 5, datetime.date(2019, 8, 1)),
 (1, 3, 6, datetime.date(2019, 8, 2)),
 (2, 7, 6, datetime.date(2019, 8, 2)),
 (3, 4, 4, datetime.date(2019, 7, 21)),
 (3, 4, 4, datetime.date(2019, 7, 21)),
 (5, 5, 5, datetime.date(2019, 7, 24)),
 (1, 1, 3, datetime.date(2019, 8, 1)),
 (1, 1, 3, datetime.date(2019, 8, 1)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2022, 8, 7)),
 (10, 11, 12, datetime.date(2020, 8, 7)),
 (10, 11, 12, datetime.date(2020, 8, 7))]

In [181]:
data[-1][-1]

datetime.date(2020, 8, 7)

### Quick note on formatting syntax:

When working with strings, you may have previously seen a `'string'`, a `"string"`, a `'''string'''`, or a `"""string"""`. While all of these are strings, the triple quotes have the added functionality of being able to use multiple lines within the same string. Sometimes, SQL queries can be much longer than others, in which case it's helpful to use new lines for readability. Here's a short example:

In [ ]:
## Uncomment cell to display contents:

# cur.execute("""SELECT * 
#                FROM employees 
#                LIMIT 5;""").fetchall()

## Wrapping Results Into Pandas DataFrames

Often, a more convenient output will be to turn these results into pandas DataFrames. To do this, you simply wrap the `c.fetchall()` output with a pandas DataFrame constructor:

In [97]:
import pandas as pd

In [101]:
cur.execute("""SELECT * FROM aws""")
df = pd.DataFrame(cur.fetchall())
df.head(10)

,0,1,2,3
0,1,3,5,2019-08-01
1,1,3,6,2019-08-02
2,2,7,7,2019-08-01
3,2,7,6,2019-08-02
4,4,7,1,2019-07-22
5,3,4,4,2019-07-21
6,3,4,4,2019-07-21
7,5,5,5,2019-07-24


Sadly as you can see this is slightly clunky as we do not have the column names. 

We can access the column names by calling `cur.description`, like so:

In [108]:
[x[0] for x in cur.description]

['article_id', 'author_id', 'viewer_id', 'view_date']

In [110]:
cur.execute("""SELECT * FROM aws""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(11)

,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
1,1,3,6,2019-08-02
2,2,7,7,2019-08-01
3,2,7,6,2019-08-02
4,4,7,1,2019-07-22
5,3,4,4,2019-07-21
6,3,4,4,2019-07-21
7,5,5,5,2019-07-24


In [111]:
df.dtypes

article_id     int64
author_id      int64
viewer_id      int64
view_date     object
dtype: object

- Check out the documentation for more info on cursor methods and attributes [here](https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor).

## The Where Clause

In general, the where clause filters query results by some condition. As you are starting to see, you can also combine multiple conditions.

### Selecting Customers From a Specific City

In [112]:
cur.execute("""SELECT * FROM aws WHERE author_id = '4';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,article_id,author_id,viewer_id,view_date
0,3,4,4,2019-07-21
1,3,4,4,2019-07-21


In [124]:
import psycopg2 
conn = psycopg2.connect(
    host = "localhost",
    database="users",
    user="postgres",
    password="postgres",
    port="5432"
)
cur = conn.cursor()

In [130]:
# cur.execute("""
#     INSERT INTO aws (article_id, author_id, viewer_id, view_date) 
#         VALUES (%s, %s, %s, %s)
# """,(5, 5, 5, '2019-07-24'))
sql_query = """
    INSERT INTO aws (article_id, author_id, viewer_id, view_date) 
    VALUES (%s,%s,%s,%s)"""
cur.execute(sql_query, (1,1,3,'2019-08-01'))
conn.commit()
# df = pd.DataFrame(cur.fetchall())
# df.columns = [x[0] for x in cur.description]
# df

### Selecting Multiple Cities

In [ ]:
cur.execute("""SELECT * FROM aws WHERE author_id = '3' OR viewer_id = '6';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

once we have the the data as a dataframe, we need to close the cursor and the connection by calling these two cods `cur.close()`, `conn.close()`

In [ ]:
import psycopg2 
conn = psycopg2.connect(
    host = "localhost",
    database="users",
    user="postgres",
    password="postgres",
    port="5432"
)
cur = conn.cursor()

In [131]:
## close the cursor
cur.close()

## close the connection 
conn.close()

### Let us now practice more using the right way

In [ ]:
## connect to postgress
conn  = psycopg2.connect(
    host = "localhost",
    database="",
    user="postgres",
    password="postgres",
    port="5432"
)
   # connection = psycopg2.connect(
    #     host = "localhost",
    #     database="users",
    #     user="postgres",
    #     password="postgres",
    #     port="5432"
    # )

### Using with 

In [ ]:
from config import config

In [ ]:
params = config()

In [152]:
connection = None
try:
    params = config()
    print('Connecting to the postgreSQL database ...')
    connection = psycopg2.connect(**params)
     # connection = psycopg2.connect(
    #     host = "localhost",
    #     database="users",
    #     user="postgres",
    #     password="postgres",
    #     port="5432"
    # )
    
    # create a cursor
    crsr = connection.cursor()
    print('PostgreSQL database version: ')
    sql_query = """
        SELECT * FROM actor
    """
    crsr.execute(sql_query)
    # db_version = crsr.fetchone()
    # print(db_version)
    df_actor = pd.DataFrame(crsr.fetchall())
    df_actor.columns = [x[0] for x in crsr.description]
    crsr.close()
except(Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if connection is not None:
        connection.commit()
        connection.close()
        print('Database connection terminated.')

{'host': 'localhost', 'database': 'users', 'user': 'postgres', 'password': 'postgres'}
Connecting to the postgreSQL database ...
PostgreSQL database version: 
Database connection terminated.


In [153]:
df_actor.head(10)

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620
5,6,Bette,Nicholson,2013-05-26 14:47:57.620
6,7,Grace,Mostel,2013-05-26 14:47:57.620
7,8,Matthew,Johansson,2013-05-26 14:47:57.620
8,9,Joe,Swank,2013-05-26 14:47:57.620
9,10,Christian,Gable,2013-05-26 14:47:57.620


In [154]:
list(df_actor[df_actor.first_name.str.startswith('P')]['first_name'])

['Penelope', 'Parker', 'Penelope', 'Penelope', 'Penelope']

In [155]:
type(df_actor.first_name.str)

pandas.core.strings.accessor.StringMethods

## Summary

In this lesson, you saw how to connect to a SQL database via python and how to subsequently execute queries against that database. Going forward, you'll continue to learn additional keywords for specifying your query parameters!